先做下准备

In [2]:
import torch

import math

首先先来看看BCE loss 的公式

$l_n = -w_n[y_n \times log(x_n) + (1-y_n) \times log(1-x_n)]$

公式是什么意思以及公式是怎么来的在我别的笔记本中有介绍
[戳这里](https://github.com/pluszeroplus/Deep-Learning/blob/master/loss/Binary%20Cross%20Entropy.pdf)
，总之先来举个例子

In [3]:
target = [0,1,0,1,1,1]
output = [0.1,0.1,0.9,0.9,0.2,0.8]

那么第一个值的loss为

In [6]:
l0 = -(target[0] * math.log(output[0]) + (1-target[0]) * math.log(1-output[0]))
l0

0.10536051565782628

第二个值的loss为

In [7]:
l1 = -(target[1] * math.log(output[1]) + (1-target[1]) * math.log(1-output[1]))
l1

2.3025850929940455

机智的你应该发现我们这两行代码是一样的！！所以这个时候写个循环遍历一下就可以搞定了！

**但是！！！**
这样不够优雅，我们可以更简单一些

首先我们先把两个变量从list类型转为torch.tensor,这样可以做一些矩阵的运算（转成numpy也行）

In [8]:
# dtype 为 float32是torch.nn.functional.binary_cross_entropy的要求
target = torch.tensor(target,dtype = torch.float32)
output = torch.tensor(output,dtype = torch.float32)

然后把同样的式子放进去

*math.log 换为torch.log是因为现在运算对象是torch.tensor了*

In [10]:
loss = -(target * torch.log(output) + (1-target) * torch.log(1-output))
loss

tensor([0.1054, 2.3026, 2.3026, 0.1054, 1.6094, 0.2231])

这不就出来了！！
稍微解释一下，式子的左边先算了positive 的loss

In [11]:
-target * torch.log(output)

tensor([0.0000, 2.3026, 0.0000, 0.1054, 1.6094, 0.2231])

右边算了negative 的loss

In [13]:
-(1-target) * torch.log(1-output)


tensor([0.1054, 0.0000, 2.3026, 0.0000, 0.0000, 0.0000])

两个相加就是最后的结果了，我们和pytorch官方的函数比对一下

In [15]:
torch.nn.functional.binary_cross_entropy(output,target,reduction = "none")

tensor([0.1054, 2.3026, 2.3026, 0.1054, 1.6094, 0.2231])

当reduction = "sum" 的时候其实就是求和

In [16]:
print(torch.sum(loss))
print(torch.nn.functional.binary_cross_entropy(output,target,reduction = "sum"))

tensor(6.6485)
tensor(6.6485)


当reduction = "mean" 的时候就是求平均数
reduction = "mean"为默认值，也就是大家一般没事的时候会用的设定

In [18]:
print(torch.mean(loss))
print(torch.nn.functional.binary_cross_entropy(output,target,reduction = "mean"))
print(torch.nn.functional.binary_cross_entropy(output,target))

tensor(1.1081)
tensor(1.1081)
tensor(1.1081)
